## Low Pass and High Pass Filtering Code

We are going to use signal processing principles to low pass filter and high pass filter our EEG data. 

The principle we will use is as follows: a convolution of two functions in the time domain is a multiplication in the frequency domain. To that end, we will convolve a low-pass filter (the most basic example being a rect function) with the EEG channels. That is, we will convolve a 1xD kernel of values 1/D over each EEG channel across the 1000 time steps. We will do the same with a high-pass filter.


*Note: We will pad the dimensions of X such that the output is the same size as the input, which means these will not be true convolutions.  

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch import Tensor
from typing import Type
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import random

cudnn.benchmark = True
plt.ion()   # interactive mode

if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

print("Using", device)

Using cuda


I am assuming the dimensions of the input are Examples x Channels(1) x Electrodes(22) x Time
The low pass filter uses a basic moving average filter (rect), which is a sinc function in the frequency domain.
The high pass filter uses a sinusoidal filter, which becomes an ideal high pass filter in the frequency domain.

In [64]:
def lpf(X, filter_width=7):
    rect = np.ones(filter_width) / filter_width
    rect = rect.reshape((1, 1, 1, rect.shape[0])) # 1x1x7
    rect = torch.tensor(rect).detach().float().to(device)
    out = nn.functional.conv2d(input=X,weight=rect,stride=1,padding='same')
    return out

def hpf(X, filter_width=7, subsample=2):
    # since the subsampling in the preprocessing affects the time axis, we need to keep track of that
    # the default subsampling in the preprocessing is 2, and the sampling rate in the paper is 250 Hz,
    # so each time step is 8 ms 
    time = np.arange(filter_width) / 250 * 2
    time = time - np.mean(time) # center time array around 0
    ihpf = 1-1/2*(1+np.cos(2*np.pi*time/filter_width))
    ihpf = ihpf.reshape((1, 1, 1, ihpf.shape[0]))
    ihpf = torch.tensor(ihpf).detach().float().to(device)
    out = nn.functional.conv2d(input=X,weight=ihpf,stride=1,padding='same')
    return out

In [69]:
# test lpf and hpf
X = np.random.normal(0.0, 1, (1, 1, 1,20))
# X = X.reshape(1, len(X))
X = torch.tensor(X).float().to(device)
print(X.size(0))
print(X.shape)
out = lpf(X)
print(out)
print(out.shape)

out2 = hpf(X)
print(out2)
print(out2.shape)

1
torch.Size([1, 1, 1, 20])
tensor([[[[-0.2488, -0.2359, -0.2046,  0.1295,  0.0531,  0.1965,  0.0719,
            0.1501,  0.2196,  0.3143,  0.0239, -0.0604, -0.2232, -0.3644,
           -0.4089, -0.5585, -0.6919, -0.7356, -0.6397, -0.5130]]]],
       device='cuda:0')
torch.Size([1, 1, 1, 20])
tensor([[[[-1.2521e-04, -2.6920e-05,  3.5601e-05,  2.8732e-04, -2.2957e-05,
           -7.0056e-05, -2.4926e-04, -4.1845e-05,  1.8565e-04,  4.0132e-04,
            7.0354e-05, -1.0913e-04, -3.2562e-04, -3.3321e-04, -7.9193e-05,
           -1.5616e-07, -4.5999e-05, -1.6670e-04, -2.4634e-04, -3.3626e-04]]]],
       device='cuda:0')
torch.Size([1, 1, 1, 20])


This makes sense because the fourier transform of a standard normal Gaussian is another Gaussian in the frequency domain. A low pass filter extracts the region close to the center, while the high pass filter extracts regions away from the center. Therefore, it makes sense that the high-pass filter produces values very close to 0.

In [67]:
# Load training data
X_train_valid = np.load("C:/Users/awong/OneDrive/Desktop/PythonScripts/C147HW/Final Project/project_data/project/X_train_valid.npy")
y_train_valid = np.load("C:/Users/awong/OneDrive/Desktop/PythonScripts/C147HW/Final Project/project_data/project/y_train_valid.npy")
y_train_valid = y_train_valid - 769
person_train_valid = np.load("C:/Users/awong/OneDrive/Desktop/PythonScripts/C147HW/Final Project/project_data/project/person_train_valid.npy")

# Load test data
X_test = np.load("C:/Users/awong/OneDrive/Desktop/PythonScripts/C147HW/Final Project/project_data/project/X_test.npy")
y_test = np.load("C:/Users/awong/OneDrive/Desktop/PythonScripts/C147HW/Final Project/project_data/project/y_test.npy")
y_test = y_test - 769
person_test = np.load("C:/Users/awong/OneDrive/Desktop/PythonScripts/C147HW/Final Project/project_data/project/person_test.npy")

# Print shapes
print('X train: ', X_train_valid.shape)
print('y train: ', y_train_valid.shape)
print('Person train+valid: ', person_train_valid.shape)
print('X test: ', X_test.shape)
print('y test: ', y_test.shape)
print('Person test: ', person_test.shape)

X train:  (2115, 22, 1000)
y train:  (2115,)
Person train+valid:  (2115, 1)
X test:  (443, 22, 1000)
y test:  (443,)
Person test:  (443, 1)


We plan to implement this after the default preprocessing. We are filtering the unprocessed data just for demonstration.

In [68]:
## Randomly split the train/validation dataset
ind_valid = np.random.choice(X_train_valid.shape[0], int(X_train_valid.shape[0]/5), replace=False)
ind_train = np.array(list(set(range(X_train_valid.shape[0])).difference(set(ind_valid))))
x_valid, y_valid = X_train_valid[ind_valid], y_train_valid[ind_valid]
x_train, y_train = X_train_valid[ind_train], y_train_valid[ind_train]
print('Training and Validation shapes before preprocessing: ')
print('X train: ', x_train.shape, ' Y train: ', y_train.shape)
print('X val: ', x_valid.shape, ' Y val', y_valid.shape)

print(x_train.shape)
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1], x_train.shape[2]))
print(x_train.shape)
x_train = torch.tensor(x_train).detach().float().to(device)
x_train_lpf = lpf(x_train)
print('Shape of training set after LPF: ', x_train_lpf.shape)

x_train_hpf = lpf(x_train)
print('Shape of training set after HPF: ', x_train_hpf.shape)

Training and Validation shapes before preprocessing: 
X train:  (1692, 22, 1000)  Y train:  (1692,)
X val:  (423, 22, 1000)  Y val (423,)
(1692, 22, 1000)
(1692, 1, 22, 1000)
Shape of training set after LPF:  torch.Size([1692, 1, 22, 1000])
Shape of training set after HPF:  torch.Size([1692, 1, 22, 1000])
